In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pathlib import Path

pd.set_option("display.max_columns", 500)

## Creating Top10 HCPCS CMS PUF file

In [2]:
CY2017 = pd.read_csv("./data/Medicare_Provider_Util_Payment_PUF_CY2017.txt", sep="\t",header=0,skiprows=[1])

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = CY2017.copy()

In [4]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt'],
      dtype='object')

In [5]:
df['Total_submitted_chrg_amt'] = df['line_srvc_cnt'] * df['average_submitted_chrg_amt']
df['Total_Medicare_payment_amt'] = df['line_srvc_cnt'] * df['average_Medicare_payment_amt']

In [6]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99217,Hospital observation care discharge,N,100.0,96,100,73.398800,325.780000,56.827200,57.492400,32578.0,5682.72
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99218,Hospital observation care typically 30 minutes,N,26.0,25,26,100.080000,449.000000,78.460000,79.306154,11674.0,2039.96
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99219,Hospital observation care typically 50 minutes,N,52.0,51,52,136.380000,614.000000,102.807692,103.895385,31928.0,5346.00
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99220,Hospital observation care typically 70 minutes...,N,59.0,59,59,190.363729,755.932203,141.293559,142.865763,44600.0,8336.32
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16,16,101.680000,462.812500,79.710000,80.750000,7405.0,1275.36


In [7]:
Total_submitted = df.Total_submitted_chrg_amt.sum()

In [8]:
#per-row (per HCPS not grouped) --(wrong code)
top = df.sort_values(by='Total_submitted_chrg_amt', ascending=False).hcpcs_code.unique() 
x = df[df.hcpcs_code.isin(top[0:10])]
x.shape

(29976, 28)

In [9]:
x.hcpcs_code.value_counts()

88305    18547
A0427     7188
G0482     1277
G0483     1225
93229      693
R0070      342
A0436      303
A0431      303
G0249       94
81528        4
Name: hcpcs_code, dtype: int64

In [10]:
# sort by top 10 hcpcs:
top_hcpcs = df.groupby(['hcpcs_code'])['Total_submitted_chrg_amt'].sum().sort_values(ascending=False)
top10_hcpcs = top_hcpcs[0:10].index
top10_hcpcs

Index(['99214', '99213', '99285', '66984', '99232', '99233', 'A0427', '99223',
       '99291', '99284'],
      dtype='object', name='hcpcs_code')

In [11]:
df_top10 = df[df.hcpcs_code.isin(top10_hcpcs)]
df_top10.shape

(1520985, 28)

In [12]:
df_top10.hcpcs_code.value_counts() #number of submitted charges
# bene_unique_cnt / total_bene_unique_cnt
# unique_npi / total_unique_npi

99213    457843
99214    428268
99232    175336
99223    133368
99233    120664
99291     62680
99284     60805
99285     57220
66984     17613
A0427      7188
Name: hcpcs_code, dtype: int64

In [13]:
pd.DataFrame({'Total_submitted_sum': top_hcpcs[1:10],\
              'Total_submitted_percent': ((top_hcpcs[0:10] / Total_submitted) * 100)\
              ''})

SyntaxError: invalid syntax (<ipython-input-13-c349d944a42f>, line 3)

In [14]:
top_hcpcs[1:10]

hcpcs_code
99213    1.287074e+10
99285    1.244005e+10
66984    8.510616e+09
99232    7.573940e+09
99233    5.667375e+09
A0427    5.432417e+09
99223    4.681324e+09
99291    4.499186e+09
99284    3.748309e+09
Name: Total_submitted_chrg_amt, dtype: float64

In [15]:
top_hcpcs[0:10] / Total_submitted * 100

hcpcs_code
99214    6.438335
99213    4.076711
99285    3.940294
66984    2.695674
99232    2.398989
99233    1.795099
A0427    1.720677
99223    1.482774
99291    1.425083
99284    1.187249
Name: Total_submitted_chrg_amt, dtype: float64

In [16]:
df_top10.groupby('hcpcs_code')['npi'].count()

hcpcs_code
66984     17613
99213    457843
99214    428268
99223    133368
99232    175336
99233    120664
99284     60805
99285     57220
99291     62680
A0427      7188
Name: npi, dtype: int64

In [17]:
#df_top10.to_csv(Path("data","top10_hcpcs.csv")) #create a new csv file

In [18]:
#sort via highest avg Total_average_submitted_charge_amount
#pick top 10 
#find proportion for the top 10
# drop all columns not in top 10

# LEIE

In [19]:
#LEIE
LEIE = pd.read_csv('data/LEIE_08_2019.csv')

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
#Top10-HCPCS
df = pd.read_csv('data/top10_hcpcs.csv', index_col=0)

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rad/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [21]:
LEIE.shape

(72476, 18)

In [22]:
LEIE['exyear'] = pd.to_datetime(LEIE["EXCLDATE"], format="%Y%m%d").dt.year
LEIE['exmonth'] = pd.to_datetime(LEIE["EXCLDATE"], format="%Y%m%d").dt.month

In [75]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,provider_type,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,Target,zip
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,False,21502
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,False,21502
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,False,21502
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,False,21502
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,False,43623


In [23]:
LEIE.columns = LEIE.columns.str.lower()
LEIE.head()

,lastname,firstname,midname,busname,general,specialty,upin,npi,dob,address,city,state,zip,excltype,excldate,reindate,waiverdate,wvrstate,exyear,exmonth
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN,1988,8
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN,1997,6
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN,2018,4
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN,2009,3
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN,1994,5


In [24]:
summary = LEIE.agg(['dtype', 'count', 'nunique', lambda x: x.isna().sum()]).rename(index={'<lambda>':'missing'}).T
summary.sort_values(by='missing', ascending=False)

,dtype,count,nunique,missing
wvrstate,object,13,10,72463
busname,object,3108,3052,69368
upin,object,6333,6156,66143
specialty,object,68276,192,4200
dob,float64,68499,20555,3977
lastname,object,69365,28194,3111
firstname,object,69366,11128,3110
address,object,72467,68551,9
state,object,72471,60,5
city,object,72475,9786,1


In [25]:
x = LEIE['npi'].value_counts()

In [26]:
x.value_counts()

1        5069
2          97
3           2
67207       1
Name: npi, dtype: int64

In [27]:
LEIE[LEIE['npi']!=0].shape

(5269, 20)

In [28]:
LEIE[LEIE['npi']==0].shape

(67207, 20)

In [29]:
LEIE.shape

(72476, 20)

In [30]:
LEIE.columns

Index(['lastname', 'firstname', 'midname', 'busname', 'general', 'specialty',
       'upin', 'npi', 'dob', 'address', 'city', 'state', 'zip', 'excltype',
       'excldate', 'reindate', 'waiverdate', 'wvrstate', 'exyear', 'exmonth'],
      dtype='object')

In [31]:
LEIE['excltype'] = LEIE['excltype'].str.strip()

## Matching npi

In [32]:
x = LEIE[LEIE['npi']!=0]
x[x.npi.duplicated()].shape #101 duplicated npis

(101, 20)

In [33]:
LEIE[LEIE.npi!=0].shape[0] #5269 npis not zero

5269

In [34]:
LEIE.npi.nunique() #5169 unique npi

5169

In [35]:
#drop all duplicated npi
LEIE_uniquenpi = LEIE.drop_duplicates(subset='npi')
LEIE_uniquenpi.shape

(5169, 20)

In [36]:
npi_same = pd.merge(df, LEIE_uniquenpi, how='inner',on='npi').npi.values

In [37]:
df['Target'] = df.npi.isin(npi_same) #add label column

In [38]:
#only 278 npi's matched!
df[df.npi.isin(npi_same)].shape

(278, 29)

In [39]:
((df.nppes_provider_country != 'US') | (df.medicare_participation_indicator == 'N')).sum()

422

In [42]:
len(df.columns)

29

In [ ]:
df.sample(5)

In [ ]:
LEIE.columns

In [43]:
((df.nppes_provider_country != 'US') | (df.medicare_participation_indicator == 'N')).sum()

422

In [44]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt', 'Total_submitted_chrg_amt',
       'Total_Medicare_payment_amt', 'Target'],
      dtype='object')

In [45]:
df = df.loc[(df.nppes_provider_country == 'US') & (df.medicare_participation_indicator == 'Y')]
dropcols = ["nppes_provider_country", "medicare_participation_indicator"]
df = df.drop(dropcols, axis=1)

In [ ]:
df.shape

In [47]:
len(df.columns)

27

In [48]:
def imputeNone(df_input):
    df = df_input.copy()
    cols = ['nppes_provider_last_org_name', 'nppes_provider_first_name', 'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender']
    for col in cols:
        df[col] = np.where(df['nppes_entity_code']=='O', "None", df[col])
    return(df)

In [49]:
df=imputeNone(df)

In [50]:
#df.to_csv(Path("data","Top10_hcpcs_labels.csv"))

## Imputing NPI in LEIE from CMS

In [51]:
# filter where npi is 0
LEIE_0 = LEIE[LEIE['npi']==0]
LEIE_0.shape

(67207, 20)

In [77]:
LEIE_0.zip.head()

0    11787
1    33125
3    11234
4    60623
5    32803
Name: zip, dtype: object

In [52]:
df['nppes_provider_zip'] = df.nppes_provider_zip.astype(str)
df['zip'] = df['nppes_provider_zip'].str[0:5]

In [53]:
df['zip'].str.len().value_counts()

5    1499175
4      19319
3       2064
1          3
2          2
Name: zip, dtype: int64

In [54]:
df['zip'] = df['zip'].apply(lambda x: x.zfill(5))

In [55]:
df['zip'].str.len().value_counts()

5    1520563
Name: zip, dtype: int64

In [56]:
LEIE_0['zip'] = LEIE_0.zip.astype(str)
LEIE_0['zip'] = LEIE_0.zip.str.slice(0,5,1)

/Users/rad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/rad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [57]:
LEIE_0['zip'].str.len().value_counts()

5    61789
4     5174
3      130
1      114
Name: zip, dtype: int64

In [58]:
LEIE_0['zip'] = LEIE_0['zip'].apply(lambda x: x.zfill(5))

/Users/rad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
LEIE_0['zip'].str.len().value_counts()

5    67207
Name: zip, dtype: int64

In [60]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'provider_type', 'place_of_service', 'hcpcs_code', 'hcpcs_description',
       'hcpcs_drug_indicator', 'line_srvc_cnt', 'bene_unique_cnt',
       'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt', 'Total_submitted_chrg_amt',
       'Total_Medicare_payment_amt', 'Target', 'zip'],
      dtype='object')

In [74]:
df.dtypes

npi                                int64
nppes_provider_last_org_name      object
nppes_provider_first_name         object
nppes_provider_mi                 object
nppes_credentials                 object
nppes_provider_gender             object
nppes_entity_code                 object
nppes_provider_street1            object
nppes_provider_street2            object
nppes_provider_city               object
nppes_provider_zip                object
nppes_provider_state              object
provider_type                     object
place_of_service                  object
hcpcs_code                        object
hcpcs_description                 object
hcpcs_drug_indicator              object
line_srvc_cnt                    float64
bene_unique_cnt                    int64
bene_day_srvc_cnt                  int64
average_Medicare_allowed_amt     float64
average_submitted_chrg_amt       float64
average_Medicare_payment_amt     float64
average_Medicare_standard_amt    float64
Total_submitted_

In [75]:
df['nppes_provider_last_org_name'] = df['nppes_provider_last_org_name'].apply(lambda x: x.lower())

AttributeError: 'float' object has no attribute 'lower'

In [61]:
df_indiv = df[df.nppes_entity_code=='I']
df_org = df[df.nppes_entity_code=='O']

In [62]:
LEIE_indiv = LEIE_0[LEIE_0.busname.isnull()]
LEIE_org = LEIE_0[LEIE_0.busname.notnull()]

In [63]:
LEIE_0.shape

(67207, 20)

In [67]:
LEIE_0 = LEIE_0.drop_duplicates()

(67187, 20)

#### Match entity code==I to Names

In [ ]:
df_indiv.shape

In [ ]:
df_org.shape

In [ ]:
LEIE_indiv.shape

In [ ]:
LEIE_org.shape

In [ ]:
df_indiv.columns

In [ ]:
df

In [ ]:
df_cols = ['nppes_provider_last_org_name', 'nppes_provider_first_name', 'nppes_provider_zip']